# Example 6.9

In [1]:
import numpy as np
import numpy.linalg as la
from IPython.display import display, Math
from scipy import stats

In [2]:
p1 = np.array([9, 6, 9])
p2 = np.array([0, 2])
p3 = np.array([3, 1, 2])

In [3]:
g1 = np.hstack([np.array([[9], [3]]),
                np.array([[6], [2]]),
                np.array([[9], [7]])])

g2 = np.hstack([np.array([[0], [4]]),
                np.array([[2], [0]])])

g3 = np.hstack([np.array([[3], [8]]),
                np.array([[1], [9]]),
                np.array([[2], [7]])])
n1, n2, n3 = g1.shape[1], g2.shape[1], g3.shape[1]
n = n1 + n2 + n3

In [4]:
# Rearrange the groups to 3D (i,j,k). The depth (i) is the group. The height (j) is the measurement. The width (k) is the observation.
x = np.stack((g1, np.hstack((g2, np.array([[np.nan], [np.nan]]))), g3))
x

array([[[ 9.,  6.,  9.],
        [ 3.,  2.,  7.]],

       [[ 0.,  2., nan],
        [ 4.,  0., nan]],

       [[ 3.,  1.,  2.],
        [ 8.,  9.,  7.]]])

In [5]:
g, p, _ = x.shape

In [6]:
# Mean for each group.
xbarg = np.stack([np.nanmean(x[i,:,:], axis=1)[:, np.newaxis] for i in range(g)])
# Global mean.
xbar = np.vstack([np.nanmean(x[:,i,:].flatten()) for i in range(p)])

In [7]:
np.nansum((x[:,0,:] - xbar[0])**2)

88.0

In [8]:
mean_matrix = np.stack([np.where(np.isnan(x[:,i,:]), np.nan, xbar[i] * np.ones((3,3))) for i in range(p)])
tr_matrix = np.stack([np.where(np.isnan(x[:,i,:]), np.nan, (xbarg[:,i,:] - xbar[i]) * np.ones((3,3))) for i in range(p)])
res_matrix = np.stack(np.stack([np.where(np.isnan(x[:,i,:]), np.nan, (x[:,i,:] - xbarg[:,i,:] * np.ones((3,3)))) for i in range(p)]))
tot_matrix = np.stack([np.where(np.isnan(x[:,i,:]), np.nan, (x[:,i,:] - xbar[i])) for i in range(p)])

In [9]:
SS_obs = np.zeros((p, p))
SS_mean = np.zeros((p, p))
SS_tr = np.zeros((p, p))
SS_res = np.zeros((p, p))
SS_tot = np.zeros((p, p))
for i in range(x.shape[1]):
    display(Math(fr'\text{{Observation }}{i+1}:'))
    display(Math(fr'''\left[
                      \begin{{array}}{{rrr}}
                            {x[0,i,0]:.0f} & {x[0,i,1]:.0f} & {x[0,i,2]:.0f} \\
                            {x[1,i,0]:.0f} & {x[1,i,1]:.0f} &            \\
                            {x[2,i,0]:.0f} & {x[2,i,1]:.0f} & {x[2,i,2]:.0f}
                      \end{{array}}
                      \right] = '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {mean_matrix[i,0,0]:.0f} & {mean_matrix[i,0,1]:.0f} & {mean_matrix[i,0,2]:.0f} \\
                            {mean_matrix[i,1,0]:.0f} & {mean_matrix[i,1,1]:.0f} &                \\
                            {mean_matrix[i,2,0]:.0f} & {mean_matrix[i,2,1]:.0f} & {mean_matrix[i,2,2]:.0f}
                      \end{{array}}
                      \right] + '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {tr_matrix[i,0,0]:.0f} & {tr_matrix[i,0,1]:.0f} & {tr_matrix[i,0,2]:.0f} \\
                            {tr_matrix[i,1,0]:.0f} & {tr_matrix[i,1,1]:.0f} &              \\
                            {tr_matrix[i,2,0]:.0f} & {tr_matrix[i,2,1]:.0f} & {tr_matrix[i,2,2]:.0f}
                      \end{{array}}
                      \right] + '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {res_matrix[i,0,0]:.0f} & {res_matrix[i,0,1]:.0f} & {res_matrix[i,0,2]:.0f} \\
                            {res_matrix[i,1,0]:.0f} & {res_matrix[i,1,1]:.0f} &               \\
                            {res_matrix[i,2,0]:.0f} & {res_matrix[i,2,1]:.0f} & {res_matrix[i,2,2]:.0f}
                      \end{{array}}
                      \right]'''

                      ))
    display(Math(r'\hspace{0.0cm}\text{(observation)}\hspace{0.9cm}\text{(mean)}\hspace{1.0cm}\text{(treatment effect)}\hspace{1.1cm}\text{(residual)}'))
    SS_obs[i,i] = (x[:,i,:][~np.isnan(x[:,i,:])]**2).sum()
    SS_mean[i,i] = np.nansum(mean_matrix[i,:,:]**2)
    SS_tr[i,i] = np.nansum(tr_matrix[i,:,:]**2)
    SS_res[i,i] = np.nansum(res_matrix[i,:,:]**2)
    SS_tot[i,i] = np.nansum(tot_matrix[i,:,:]**2)
    display(Math(r'\text{and}'))
    display(Math(r'\text{SS}_{\text{obs}} = \text{SS}_{\text{mean}} + \text{SS}_{\text{tr}} + \text{SS}_{\text{res}}'))
    display(Math(fr'{SS_obs[i,i]:.0f} = {SS_mean[i,i]:.0f} + {SS_tr[i,i]:.0f} + {SS_res[i,i]:.0f}'))
    display(Math(fr'\text{{Total SS (corrected)}} = \text{{SS}}_{{\text{{obs}}}} - \text{{SS}}_{{\text{{mean}}}} = '
                 f'{SS_obs[i,i]:.0f} - {SS_mean[i,i]:.0f} = {SS_tot[i,i]:.0f}'))
    print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
# Fill in the cross products.
SS_mean[0,1] = SS_mean[1,0] = (np.nan_to_num(mean_matrix[0,:,:]) * np.nan_to_num(mean_matrix[1,:,:])).sum()
SS_tr[0,1] = SS_tr[1,0] = (np.nan_to_num(tr_matrix[0,:,:]) * np.nan_to_num(tr_matrix[1,:,:])).sum()
SS_res[0,1] = SS_res[1,0] = (np.nan_to_num(res_matrix[0,:,:]) * np.nan_to_num(res_matrix[1,:,:])).sum()
SS_tot[0,1] = SS_tot[1,0] = (np.nan_to_num(tot_matrix[0,:,:]) * np.nan_to_num(tot_matrix[1,:,:])).sum()

In [11]:
display(Math(r'\begin{array}{lll}'
             r'\text{Source} & \text{Matrix of sum of squares} &  \\'
             r'\text{of variation} & \text{and cross products} & \text{Degrees of freedom} \\'
             r'\hline \\'
             r'\text{Treatment} &'
             r' \left[ \begin{array}{rr}'
             fr' {SS_tr[0,0]:.0f} & {SS_tr[0,1]:.0f} \\'
             fr' {SS_tr[1,0]:.0f} & {SS_tr[1,1]:.0f}'
             r' \end{array} \right] &'
             fr'{g} - 1 = {g - 1} \\ \\'
             r'\text{Residual} &'
             r' \left[ \begin{array}{rr}'
             fr' \phantom{{-}} {SS_res[0,0]:.0f} & \phantom{{-}} {SS_res[0,1]:.0f} \\'
             fr' \phantom{{-}} {SS_res[1,0]:.0f} & \phantom{{-}} {SS_res[1,1]:.0f}'
             r' \end{array} \right] &'
             fr'{n1} + {n2} + {n3} - {g} = {n - g} \\ \\'
             r'\hline \\'
             r'\text{Total (corrected)} &'
             r' \left[ \begin{array}{rr}'
             fr' {SS_tot[0,0]:.0f} & {SS_tot[0,1]:.0f} \\'
             fr' {SS_tot[1,0]:.0f} & {SS_tot[1,1]:.0f}'
             r' \end{array} \right] &'
             f'{(n - 1)}'
             r'\end{array}'
             ))

<IPython.core.display.Math object>

In [12]:
np.all(np.equal(SS_tot, SS_tr + SS_res))

True

In [13]:
display(Math(r' \begin{bmatrix}'
             fr' {SS_tr[0,0]:.0f} & {SS_tr[0,1]:.0f} \\'
             fr' {SS_tr[1,0]:.0f} & {SS_tr[1,1]:.0f}'
             r' \end{bmatrix}'
             '='
             r' \begin{bmatrix}'
             fr' {SS_tr[0,0]:.0f} & {SS_tr[0,1]:.0f} \\'
             fr' {SS_tr[1,0]:.0f} & {SS_tr[1,1]:.0f}'
             r' \end{bmatrix}'
             '+'
             r' \begin{bmatrix}'
             fr' {SS_res[0,0]:.0f} & {SS_res[0,1]:.0f} \\'
             fr' {SS_res[1,0]:.0f} & {SS_res[1,1]:.0f}'
             r' \end{bmatrix}'
             ))

<IPython.core.display.Math object>

In [14]:
lmbda_star = la.det(SS_res)/la.det(SS_tot)

In [15]:
display(Math(r'\Lambda^{\star}'
             '='
             r'\frac{\left|\textbf{W}\right|}{\left|\textbf{B} + \textbf{W}\right|}'
             '='
             fr'\frac{{{SS_res[0,0]:.0f}({SS_res[1,1]:.0f}) - ({SS_res[0,1]:.0f})^{{2}} }}{{{SS_tot[0,0]:.0f}({SS_tot[1,1]:.0f}) - ({SS_tot[0,1]:.0f})^{{2}} }}'
             '='
             fr'\frac{{{la.det(SS_res):.0f}}}{{{la.det(SS_tr + SS_res):.0f}}}'
             '='
             f'{lmbda_star:.4f}'
             ))

<IPython.core.display.Math object>

In [16]:
display(Math(fr'\text{{Since}} \hspace{{0.2cm}} p = {p}'
             fr'\hspace{{0.2cm}} \text{{and}} \hspace{{0.2cm}} '
             fr'g = {g} \hspace{{0.2cm}} \text{{using Table 6.3}},'
             r'\left( \frac{ \sum n_{\ell} - g - 1 }{ g - 1 } \right)'
             r'\left( \frac{ 1 - \sqrt{\Lambda^{\star}} }{ \sqrt{\Lambda^{\star}} } \right)'
             r'\sim F_{2(g-1), 2(\sum n_{\ell} - g - 1)}'
             ))
# 

<IPython.core.display.Math object>

In [17]:
test_stat = ((n - g - 1)/(g-1)) * (1 - np.sqrt(lmbda_star))/np.sqrt(lmbda_star)
alpha = 0.01
f_crit = stats.f.ppf(1-alpha, dfn=2*(g-1), dfd=2*(n-g-1))

In [18]:
display(Math(r'F^{\star} = \left( \frac{ \sum n_{\ell} - g - 1 }{ g - 1 } \right)'
             r'\left( \frac{ 1 - \sqrt{\Lambda^{\star}} }{ \sqrt{\Lambda^{\star}} } \right)'
             '='
             fr'\left( \frac{{ {n} - {g} - 1 }}{{ {g} - 1 }} \right)'
             fr'\left( \frac{{ 1 - \sqrt{{ {lmbda_star:.4f} }} }}{{ \sqrt{{ {lmbda_star:.4f} }} }} \right)'
             '='
             f'{test_stat:.4f}'
             ))

<IPython.core.display.Math object>

In [19]:
display(Math(fr'F_{{\text{{crit}}}} = F_{{2(g-1), 2(\sum n_{{\ell}} - g - 1)}}'
             '='
             fr'F_{{2({g}-1), 2({n} - {g} - 1)}} \left( {alpha} \right)'
             '='
             fr'F_{{{2*(g-1)}, {2*(n-g-1)} }} \left( {alpha} \right)'
             '='
             f'{f_crit:.3f}'
             ))

<IPython.core.display.Math object>

In [20]:
if test_stat > f_crit:
    display(Math(fr'\text{{We have that }} F^{{\star}} = {test_stat:.3f} > F_{{\text{{crit}}}} = F_{{{2*(g-1)}, {2*(n-g-1)} }} \left( {alpha} \right) = '
                 fr'{f_crit:.3f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\tau}_{1} = \bm{\tau}_{3} = \bm{\tau}_{3} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F^{{\star}} = {test_stat:.3f} \leq F_{{\text{{crit}}}} = F_{{{2*(g-1)}, {2*(n-g-1)} }} \left( {alpha} \right) = '
                fr'{f_crit:.3f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\tau}_{1} = \bm{\tau}_{3} = \bm{\tau}_{3} = \textbf{0}'))

<IPython.core.display.Math object>